In [1]:
#今回取得する情報; 物件名　住所　間取り　家賃　階数
#前提条件; 通勤時間が東京駅から20分になる物件

#基本
import pandas as pd
import numpy as up
import requests

#Selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager

#Beautifulsoup(一応)
from bs4 import BeautifulSoup

#request(一応)
import urllib.request as req

#time
from time import sleep
from tqdm import tqdm

#固定情報
URL = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&ta=13&bs=040&ekInput=25620&tj=20&nk=-1&ct=9999999&cb=0.0&et=9999999&mt=9999999&mb=0&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&pc=30'

#time_start = time.time()        
#検索(ブラウザ非表示)
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(options = options)
sleep(2)
        
browser.get(URL)
sleep(5)
page_num = int(browser.find_elements(By.TAG_NAME, 'ol')[1].text.split()[-1])

#データ格納用
bukken_uppers = []
bukken_names = []
bukken_addresses = []
bukken_status1s = []
bukken_status2s = []
bukken_status3s = []
bukken_years = []
bukken_totalfloors = []
bukken_floors = []
bukken_rents = []
bukken_managementfees = []
bukken_deposits = []
bukken_granuityfees = []
bukken_houselayouts = []
bukken_exclusiveareas = []

#今回は数が多いので一旦5ページ分
for l in tqdm(range(5)):
    l = l + 1
    _URL = URL + '&page=' + str(l)
    
    browser.get(_URL)
    sleep(5)
    bukken_list = browser.find_element(By.ID, 'js-bukkenList')
    
    for i in range(0, len(bukken_list.find_elements(By.CLASS_NAME, 'l-cassetteitem'))):
        bukken_lists = bukken_list.find_elements(By.CLASS_NAME, 'l-cassetteitem')[i]
        for k in range(0, len(bukken_list.find_elements(By.CLASS_NAME, 'cassetteitem'))):
            bukken = bukken_list.find_elements(By.CLASS_NAME, 'cassetteitem')[k]
                    
            #上部分
            bukken_upper = bukken.find_element(By.CLASS_NAME, 'cassetteitem-detail')
            bukken_name = bukken_upper.find_element(By.CLASS_NAME, 'cassetteitem_content-title').text
            bukken_address = bukken_upper.find_element(By.CLASS_NAME, 'cassetteitem_detail-col1').text
            bukken_status = bukken_upper.find_element(By.CLASS_NAME, 'cassetteitem_detail-col2').text.split('\n')
            bukken_status1 = bukken_status[0]
            bukken_status2 = bukken_status[1] if len(bukken_status) == 3 else None
            bukken_status3 = bukken_status[2] if len(bukken_status) == 3 else None
            bukken_year, bukken_totalfloor = bukken_upper.find_element(By.CLASS_NAME, 'cassetteitem_detail-col3').text.split('\n')
            #print(bukken_upper)
            #print(bukken_name)
            #print(bukken_address)
            #print(bukken_status1)
            #print(bukken_status2)
            #print(bukken_status3)
            #print(bukken_year)
            #print(bukken_totalfloor)
        
            #下部分
            bukken_bottom = bukken.find_element(By.CLASS_NAME, 'cassetteitem-item')
            for j in range(0, len(bukken_bottom.find_elements(By.TAG_NAME, 'tbody'))):
                bukken_item = bukken_bottom.find_elements(By.TAG_NAME, 'tbody')[j]
                bukken_floor = bukken_item.find_elements(By.TAG_NAME, 'td')[2].text
                bukken_rent, bukken_managementfee = bukken_item.find_elements(By.TAG_NAME, 'td')[3].text.split('\n')
                bukken_deposit, bukken_gratuityfee = bukken_item.find_elements(By.TAG_NAME, 'td')[4].text.split('\n')
                bukken_houselayout, bukken_exclusivearea = bukken_item.find_elements(By.TAG_NAME, 'td')[5].text.split('\n')
                #print(bukken_floor)
                #print(bukken_rent)
                #print(bukken_managementfee)
                #print(bukken_deposit)
                #print(bukken_gratuityfee)
                #print(bukken_houselayout)
                #print(bukken_exclusivearea)
            
                bukken_names.append(bukken_name)
                bukken_addresses.append(bukken_address)
                bukken_status1s.append(bukken_status1)
                bukken_status2s.append(bukken_status2)
                bukken_status3s.append(bukken_status3)
                bukken_years.append(bukken_year)
                bukken_totalfloors.append(bukken_totalfloor)
                bukken_floors.append(bukken_floor)
                bukken_rents.append(bukken_rent)
                bukken_managementfees.append(bukken_managementfee)
                bukken_deposits.append(bukken_deposit)
                bukken_granuityfees.append(bukken_gratuityfee)
                bukken_houselayouts.append(bukken_houselayout)
                bukken_exclusiveareas.append(bukken_exclusivearea)
                        
df = pd.DataFrame({
    'Name': bukken_names,
    'Address': bukken_addresses,
    'Status1': bukken_status1s,
    'Status2': bukken_status2s,
    'Status3': bukken_status3s,
    'Year': bukken_years,
    'Total Floor': bukken_totalfloors,
    'Floor': bukken_floors,
    'Rent': bukken_rents,
    'Management Fee': bukken_managementfees,
    'Deposit': bukken_deposits,
    'Granuity Fee': bukken_granuityfees,
    'House Layout': bukken_houselayouts,
    'Exclusive Area': bukken_exclusiveareas
})

100%|█████████████████████████████████████████████| 5/5 [03:07<00:00, 37.58s/it]


In [2]:
df

,Name,Address,Status1,Status2,Status3,Year,Total Floor,Floor,Rent,Management Fee,Deposit,Granuity Fee,House Layout,Exclusive Area
0,ＪＲ京葉線 葛西臨海公園駅 8階建 築11年,東京都江戸川区南葛西４,ＪＲ京葉線/葛西臨海公園駅 歩21分,ＪＲ京葉線/舞浜駅 歩38分,東京メトロ東西線/葛西駅 歩23分,築11年,8階建,8階,15万円,9000円,15万円,15万円,2LDK,60.3m2
1,フラット谷中,東京都台東区谷中３,東京メトロ千代田線/千駄木駅 歩3分,ＪＲ山手線/日暮里駅 歩10分,都営三田線/白山駅 歩17分,築35年,地下1地上3階建,3階,7万円,5000円,-,-,ワンルーム,19.7m2
2,広尾ガーデンヒルズM棟,東京都渋谷区広尾４,東京メトロ日比谷線/広尾駅 歩6分,都営大江戸線/六本木駅 歩20分,東京メトロ千代田線/乃木坂駅 歩21分,築40年,地下1地上7階建,1階,49万円,-,147万円,49万円,2LDK,64.61m2
3,広尾ガーデンヒルズM棟,東京都渋谷区広尾４,東京メトロ日比谷線/広尾駅 歩6分,都営大江戸線/六本木駅 歩20分,東京メトロ千代田線/乃木坂駅 歩21分,築40年,地下1地上7階建,5階,63万円,-,126万円,63万円,4LDK,108.95m2
4,広尾ガーデンヒルズM棟,東京都渋谷区広尾４,東京メトロ日比谷線/広尾駅 歩6分,都営大江戸線/六本木駅 歩20分,東京メトロ千代田線/乃木坂駅 歩21分,築40年,地下1地上7階建,5階,65万円,-,195万円,65万円,2LDK,85.51m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,東京メトロ千代田線 湯島駅 12階建 築20年,東京都文京区湯島３,東京メトロ千代田線/湯島駅 歩1分,東京メトロ銀座線/上野広小路駅 歩5分,ＪＲ山手線/御徒町駅 歩7分,築20年,12階建,9階,8.5万円,10000円,8.5万円,-,1K,21.45m2
3128,東京メトロ千代田線 湯島駅 12階建 築20年,東京都文京区湯島３,東京メトロ千代田線/湯島駅 歩1分,東京メトロ銀座線/上野広小路駅 歩5分,ＪＲ山手線/御徒町駅 歩7分,築20年,12階建,10階,9万円,10000円,9万円,-,1K,21.29m2
3129,モーニングサイド銀座,東京都中央区京橋３,東京メトロ銀座線/京橋駅 歩2分,東京メトロ有楽町線/銀座一丁目駅 歩7分,ＪＲ京葉線/八丁堀駅 歩8分,築22年,13階建,3階,9.6万円,8000円,9.6万円,9.6万円,1K,20.28m2
3130,モーニングサイド銀座,東京都中央区京橋３,東京メトロ銀座線/京橋駅 歩2分,東京メトロ有楽町線/銀座一丁目駅 歩7分,ＪＲ京葉線/八丁堀駅 歩8分,築22年,13階建,3階,9.9万円,8000円,9.9万円,9.9万円,1K,20.28m2
